In [1]:
# import tensorflow as tf
import pandas as pd
import numpy as np
from scipy.stats import zscore

from pandas import read_csv 
# from keras.models import Sequential 
# from keras.layers import Dense 
# from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [ ]:
# print(tf.__version__)

In [ ]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install scipy --user

In [5]:
# reading dataset created by Arun
df = pd.read_csv("dataset_exported.csv", sep=";")
# eliminating columns 'Quality_x'
df = df.iloc[:,:61].copy(deep=True)

In [6]:
# create unique set of well locations - will be used later as a identifiers
df_uni = (df[['lng', 'lat']].drop_duplicates().reset_index())[['lng', 'lat']]
df_uni['well_loc'] = df_uni.index.values
df = pd.merge(df_uni, df, how='left', on=['lng', 'lat'])

In [7]:
# drop 'id' column from dataset
df.drop('id', axis=1, inplace=True)
df.shape

(10205, 61)

In [8]:
# show a sample of the 'horizontal' observations
df.reset_index()
df[['well_loc', 'lng', 'lat', 'DEPT', 'DPHI', 'GR', 'ILD', 'NPHI', 'HorID', 'DPHI1', 'NPHI_near_3']].tail()

,well_loc,lng,lat,DEPT,DPHI,GR,ILD,NPHI,HorID,DPHI1,NPHI_near_3
10200,1236,-111.198275,56.629919,72.0,0.256,75.364,4.308,0.448,10000,0.300,0.403
10201,1236,-111.198275,56.629919,76.5,0.266,79.086,8.597,0.402,11000,0.292,0.386
10202,1236,-111.198275,56.629919,76.5,0.266,79.086,8.597,0.402,12000,0.287,0.409
10203,1236,-111.198275,56.629919,76.5,0.266,79.086,8.597,0.402,13000,0.276,0.400
10204,1236,-111.198275,56.629919,136.0,0.188,99.505,24.309,0.302,14000,0.285,0.411


In [9]:
df[['well_loc', 'lng', 'lat', 'DEPT', 'DPHI', 'GR', 'ILD', 'NPHI', 'HorID', 'DPHI1', 'NPHI_near_3']].head()

,well_loc,lng,lat,DEPT,DPHI,GR,ILD,NPHI,HorID,DPHI1,NPHI_near_3
0,0,-114.620656,55.288691,393.0,0.299,72.249,3.338,0.429,1000,0.215,0.425
1,0,-114.620656,55.288691,549.0,0.247,76.644,4.313,0.427,4000,0.226,0.606
2,0,-114.620656,55.288691,584.0,0.184,72.472,5.379,0.357,5000,0.210,0.416
3,0,-114.620656,55.288691,597.0,0.180,69.942,4.265,0.367,7000,0.528,0.413
4,0,-114.620656,55.288691,607.0,0.191,57.457,9.279,0.312,11000,0.150,0.369


In [18]:
df.shape

(10205, 61)

In [10]:
# calculate zscore for every column in a dataframe
def make_zscore(columns = [], df = pd.DataFrame()):
    ds = pd.DataFrame()
    for col in columns:
        ds[col] = df[col].to_frame().apply(zscore, ddof=1)
    return(ds)

def verticalize_ds(anchor_col = 'DEPT', value = 'reading', ds = pd.DataFrame()):
    # verticalize dataset by log observation
    data = pd.DataFrame()
    data = (ds.replace(0, np.nan)
              .set_index(anchor_col, append=True)
              .stack()
              .reset_index()
              .rename(columns={0:value}))
    data.columns = ['p_idx', anchor_col, 'CurveType', value]
    return(data)

## Create Z-Score Values - dataset 'ds'

In [11]:
# create zscore values for numerical log columns
# log_columns = ['DEPT', 'DPHI', 'GR', 'ILD', 'NPHI', 
#            'DPHI1', 'GR1', 'ILD1', 'NPHI1', 'DPHI2', 'GR2', 'ILD2', 'NPHI2', 
#            'DPHI3', 'GR3', 'ILD3', 'NPHI3', 'DPHI4', 'GR4', 'ILD4', 'NPHI4', 
#            'DPHI5', 'GR5', 'ILD5', 'NPHI5', 'DPHI6', 'GR6', 'ILD6', 'NPHI6', 
#            'DPHI7', 'GR7', 'ILD7', 'NPHI7', 'DPHI8', 'GR8', 'ILD8', 'NPHI8', 
#            'DPHI9', 'GR9', 'ILD9', 'NPHI9', 'DPHI10', 'GR10', 'ILD10', 'NPHI10', 
#            'DPHI_near_1', 'GR_near_1', 'ILD_near_1', 'NPHI_near_1', 
#            'DPHI_near_2', 'GR_near_2', 'ILD_near_2', 'NPHI_near_2', 
#            'DPHI_near_3', 'GR_near_3', 'ILD_near_3', 'NPHI_near_3']
log_columns = ['DPHI', 'GR', 'ILD', 'NPHI', 
           'DPHI1', 'GR1', 'ILD1', 'NPHI1', 'DPHI2', 'GR2', 'ILD2', 'NPHI2', 
           'DPHI3', 'GR3', 'ILD3', 'NPHI3', 'DPHI4', 'GR4', 'ILD4', 'NPHI4', 
           'DPHI5', 'GR5', 'ILD5', 'NPHI5', 'DPHI6', 'GR6', 'ILD6', 'NPHI6', 
           'DPHI7', 'GR7', 'ILD7', 'NPHI7', 'DPHI8', 'GR8', 'ILD8', 'NPHI8', 
           'DPHI9', 'GR9', 'ILD9', 'NPHI9', 'DPHI10', 'GR10', 'ILD10', 'NPHI10', 
           'DPHI_near_1', 'GR_near_1', 'ILD_near_1', 'NPHI_near_1', 
           'DPHI_near_2', 'GR_near_2', 'ILD_near_2', 'NPHI_near_2', 
           'DPHI_near_3', 'GR_near_3', 'ILD_near_3', 'NPHI_near_3']
ds = make_zscore(log_columns, df)
ds.tail()

,DPHI,GR,ILD,NPHI,DPHI1,GR1,ILD1,NPHI1,DPHI2,GR2,...,ILD_near_1,NPHI_near_1,DPHI_near_2,GR_near_2,ILD_near_2,NPHI_near_2,DPHI_near_3,GR_near_3,ILD_near_3,NPHI_near_3
10200,0.376638,-0.437644,-0.104629,0.343128,0.881925,0.187726,-0.105918,0.915366,0.663671,0.156417,...,-0.033659,0.815322,-0.898025,1.331451,-0.117951,-0.483191,-0.351046,0.744098,-0.116891,0.008416
10201,0.501542,-0.276749,-0.041809,-0.199568,0.779123,-0.277090,-0.003720,-0.364493,0.715811,-0.206354,...,-0.033659,0.815322,-0.536226,1.633602,-0.136072,1.300190,-0.821500,1.249115,-0.129169,-0.205680
10202,0.501542,-0.276749,-0.041809,-0.199568,0.714871,-0.171718,-0.017388,-0.471148,0.572427,-0.126527,...,0.120310,-2.381857,0.137469,0.291827,-0.068453,0.082572,0.230832,0.251362,-0.112027,0.083979
10203,0.501542,-0.276749,-0.041809,-0.199568,0.573518,-0.092286,-0.043383,-0.352642,0.481182,-0.020631,...,-0.125707,0.549853,0.137469,0.291827,-0.068453,0.082572,-0.648175,0.533881,-0.105230,-0.029366
10204,-0.472706,0.605930,0.188318,-1.379344,0.689171,-0.941338,0.633691,-0.293389,0.416007,-1.030058,...,-0.125707,0.549853,0.137469,0.291827,-0.068453,0.082572,-0.920543,0.762174,-0.110549,0.109167


## Merge on a left join based on indexes - dataset _'dt'_

In [12]:
# merge left join on indexes
dt = pd.merge(df[['well_loc', 'lng', 'lat', 'HorID', 'DEPT']], ds, how='left', left_index=True, right_index=True)
dt[['well_loc', 'lng', 'lat', 'DPHI', 'GR', 'ILD', 'NPHI', 'HorID', 'DPHI1', 'NPHI_near_3']].tail()

,well_loc,lng,lat,DPHI,GR,ILD,NPHI,HorID,DPHI1,NPHI_near_3
10200,1236,-111.198275,56.629919,0.376638,-0.437644,-0.104629,0.343128,10000,0.881925,0.008416
10201,1236,-111.198275,56.629919,0.501542,-0.276749,-0.041809,-0.199568,11000,0.779123,-0.205680
10202,1236,-111.198275,56.629919,0.501542,-0.276749,-0.041809,-0.199568,12000,0.714871,0.083979
10203,1236,-111.198275,56.629919,0.501542,-0.276749,-0.041809,-0.199568,13000,0.573518,-0.029366
10204,1236,-111.198275,56.629919,-0.472706,0.605930,0.188318,-1.379344,14000,0.689171,0.109167


In [13]:
dt.shape

(10205, 61)

In [17]:
dt.columns

Index(['well_loc', 'lng', 'lat', 'HorID', 'DEPT', 'DPHI', 'GR', 'ILD', 'NPHI',
       'DPHI1', 'GR1', 'ILD1', 'NPHI1', 'DPHI2', 'GR2', 'ILD2', 'NPHI2',
       'DPHI3', 'GR3', 'ILD3', 'NPHI3', 'DPHI4', 'GR4', 'ILD4', 'NPHI4',
       'DPHI5', 'GR5', 'ILD5', 'NPHI5', 'DPHI6', 'GR6', 'ILD6', 'NPHI6',
       'DPHI7', 'GR7', 'ILD7', 'NPHI7', 'DPHI8', 'GR8', 'ILD8', 'NPHI8',
       'DPHI9', 'GR9', 'ILD9', 'NPHI9', 'DPHI10', 'GR10', 'ILD10', 'NPHI10',
       'DPHI_near_1', 'GR_near_1', 'ILD_near_1', 'NPHI_near_1', 'DPHI_near_2',
       'GR_near_2', 'ILD_near_2', 'NPHI_near_2', 'DPHI_near_3', 'GR_near_3',
       'ILD_near_3', 'NPHI_near_3', 'p_idx'],
      dtype='object')

In [14]:
dt['p_idx'] = dt.index.values
dt_columns = ['p_idx', 'well_loc', 'lng', 'lat', 'HorID', 'DEPT', 'DPHI', \
              'GR', 'ILD', 'NPHI', 'HorID', 'DPHI1', 'NPHI_near_3']
dt[dt_columns].tail()

,p_idx,well_loc,lng,lat,HorID,DEPT,DPHI,GR,ILD,NPHI,HorID,DPHI1,NPHI_near_3
10200,10200,1236,-111.198275,56.629919,10000,72.0,0.376638,-0.437644,-0.104629,0.343128,10000,0.881925,0.008416
10201,10201,1236,-111.198275,56.629919,11000,76.5,0.501542,-0.276749,-0.041809,-0.199568,11000,0.779123,-0.205680
10202,10202,1236,-111.198275,56.629919,12000,76.5,0.501542,-0.276749,-0.041809,-0.199568,12000,0.714871,0.083979
10203,10203,1236,-111.198275,56.629919,13000,76.5,0.501542,-0.276749,-0.041809,-0.199568,13000,0.573518,-0.029366
10204,10204,1236,-111.198275,56.629919,14000,136.0,-0.472706,0.605930,0.188318,-1.379344,14000,0.689171,0.109167


In [16]:
dt[dt_columns].head()

,p_idx,well_loc,lng,lat,HorID,DEPT,DPHI,GR,ILD,NPHI,HorID,DPHI1,NPHI_near_3
0,0,0,-114.620656,55.288691,1000,393.0,0.913723,-0.572301,-0.118836,0.118971,1000,-0.210350,0.285482
1,1,0,-114.620656,55.288691,4000,549.0,0.264225,-0.382312,-0.104555,0.095375,4000,-0.068997,2.564976
2,2,0,-114.620656,55.288691,5000,584.0,-0.522668,-0.562661,-0.088942,-0.730467,5000,-0.274602,0.172137
3,3,0,-114.620656,55.288691,7000,597.0,-0.572629,-0.672028,-0.105258,-0.612490,7000,3.811793,0.134355
4,4,0,-114.620656,55.288691,11000,607.0,-0.435235,-1.211734,-0.031820,-1.261366,11000,-1.045620,-0.419776


In [ ]:
# verticalize logs data stored on variable 'columns'
data = verticalize_ds('DEPT', 'reading', dt[log_columns])
data.tail()

In [ ]:
# left outer join to HorID and lat and long coordinates on DEPT
df_out = pd.merge(data, dt, how='left', on=['p_idx'])
df_out.rename(index=str, columns={"DEPT_x": "DEPT"}, inplace=True)
df_out.drop(['p_idx'], inplace=True, axis=1)
df_out.tail()

In [ ]:
df_out.reset_index(drop=True, inplace=True)
# df_out.index.values
df_out.shape

In [ ]:
# One Hot Encoding HorID
Y = df_out['CurveType']
Y.head()
# encode class values as integers 
encoder = LabelEncoder() 
encoder.fit(Y) 
encoded_Y = encoder.transform(Y) 
# convert integers to dummy variables (i.e. one hot encoded) 
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y.shape

In [ ]:
curves = df_out['CurveType'].unique()
curves

In [ ]:
dummy_cols = ['cType_' + x for x in curves]
df_out_dummy = pd.DataFrame(dummy_y, columns=dummy_cols)
df_out_dummy.columns

In [ ]:
df_out = pd.merge(df_out, df_out_dummy, how='left', left_index=True, right_index=True).copy(deep=True)

In [ ]:
# pd.set_option('display.width', 240)
cols_to_drop = ['CurveType', 'well_loc', 'DEPT_y','DPHI', 'GR', 'ILD', 'NPHI', 'DPHI1', 'GR1', 'ILD1', 'NPHI1', 'DPHI2', 'GR2', 'ILD2', 'NPHI2', 'DPHI3', 'GR3', 'ILD3', 'NPHI3', 'DPHI4', 'GR4', 'ILD4',
       'NPHI4', 'DPHI5', 'GR5', 'ILD5', 'NPHI5', 'DPHI6', 'GR6', 'ILD6', 'NPHI6', 'DPHI7', 'GR7', 'ILD7', 'NPHI7', 'DPHI8', 'GR8', 'ILD8', 'NPHI8', 'DPHI9', 'GR9', 'ILD9', 'NPHI9', 'DPHI10', 'GR10', 'ILD10', 'NPHI10', 'DPHI_near_1',
       'GR_near_1', 'ILD_near_1', 'NPHI_near_1', 'DPHI_near_2', 'GR_near_2', 'ILD_near_2', 'NPHI_near_2', 'DPHI_near_3', 'GR_near_3', 'ILD_near_3', 'NPHI_near_3']
df_out.drop(cols_to_drop, inplace=True, axis=1)
df_out.head()

In [ ]:
df_out[['lng', 'lat']] = make_zscore(['lng', 'lat'], df_out[['lng', 'lat']])

In [ ]:
df_out.tail()

In [ ]:
df_out.HorID.unique()

In [ ]:
mask = df_out['HorID'].isin([1000, 5000,  7000, 11000, 12000, 13000, 14000, 
                             9000,  9500, 10000])
df_out = df_out[mask].copy(deep=True)
df_out.tail()

In [ ]:
df_out.to_csv(path_or_buf='dataset_normalized_20180222_0241.csv', index=False)